In [1]:
import pandas as pd
import pgeocode
from geopy.geocoders import Nominatim
import numpy as np

In [2]:
users = pd.read_csv('./Data/users.csv')
cards = pd.read_csv('./Data/cards.csv')
trans = pd.read_csv('./Data/transactions.csv')

In [3]:
users.shape, cards.shape, trans.shape

((2000, 19), (6146, 13), (24386900, 15))

In [4]:
users.columns

Index(['User', 'Person', 'Current Age', 'Retirement Age', 'Birth Year',
       'Birth Month', 'Gender', 'Address', 'Apartment', 'City', 'State',
       'Zipcode', 'Latitude', 'Longitude', 'Per Capita Income - Zipcode',
       'Yearly Income - Person', 'Total Debt', 'FICO Score',
       'Num Credit Cards'],
      dtype='object')

In [5]:
cards.columns

Index(['User', 'CARD INDEX', 'Card Brand', 'Card Type', 'Card Number',
       'Expires', 'CVV', 'Has Chip', 'Cards Issued', 'Credit Limit',
       'Acct Open Date', 'Year PIN last Changed', 'Card on Dark Web'],
      dtype='object')

In [8]:
trans.columns

Index(['User', 'Card', 'Year', 'Month', 'Day', 'Time', 'Amount', 'Use Chip',
       'Merchant Name', 'Merchant City', 'Merchant State', 'Zip', 'MCC',
       'Errors?', 'Is Fraud?'],
      dtype='object')

# Geocode Example

In [27]:
geolocator = Nominatim(user_agent="geoapiExercises")
location = geolocator.geocode('91750')

In [15]:
location

Location(Île-de-France, France métropolitaine, 91750, France, (48.5150632, 2.4419139839231048, 0.0))

In [16]:
location[-1]

(48.5150632, 2.4419139839231048)

# Pgeocode Example

In [40]:
pgeo = pgeocode.Nominatim('us')

In [41]:
pgeo.query_postal_code('91750')

postal_code             91750
country_code               US
place_name           La Verne
state_name         California
state_code                 CA
county_name       Los Angeles
county_code              37.0
community_name           None
community_code            NaN
latitude              34.1159
longitude           -117.7708
accuracy                  4.0
Name: 0, dtype: object

# NaN Counts

In [13]:
trans.isna().sum()

User                     0
Card                     0
Year                     0
Month                    0
Day                      0
Time                     0
Amount                   0
Use Chip                 0
Merchant Name            0
Merchant City            0
Merchant State     2720821
Zip                2878135
MCC                      0
Errors?           23998469
Is Fraud?                0
dtype: int64

# Handle Online Transaction

In [8]:
online = trans[trans['Merchant City'] == 'ONLINE']
online

,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?
11,0,0,2002,9,5,20:41,$53.91,Online Transaction,-9092677072201095172,ONLINE,NaN,NaN,4900,NaN,No
24,0,0,2002,9,9,20:02,$144.90,Online Transaction,-8338381919281017248,ONLINE,NaN,NaN,4899,NaN,No
85,0,0,2002,9,30,06:21,$127.32,Online Transaction,-7421093378627544099,ONLINE,NaN,NaN,5311,NaN,No
99,0,0,2002,10,6,06:14,$139.39,Online Transaction,-7421093378627544099,ONLINE,NaN,NaN,5311,NaN,No
106,0,0,2002,10,9,08:16,$53.09,Online Transaction,-4956618006720593695,ONLINE,NaN,NaN,5193,NaN,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24386877,1999,1,2020,2,24,20:04,$55.79,Online Transaction,-6160036380778658394,ONLINE,NaN,NaN,4121,NaN,No
24386879,1999,1,2020,2,25,07:06,$43.08,Online Transaction,-6160036380778658394,ONLINE,NaN,NaN,4121,NaN,No
24386880,1999,1,2020,2,25,07:34,$43.76,Online Transaction,-6160036380778658394,ONLINE,NaN,NaN,4121,NaN,No
24386884,1999,1,2020,2,26,07:43,$45.18,Online Transaction,-6160036380778658394,ONLINE,NaN,NaN,4121,NaN,No


In [14]:
def map_lat_from_users(id):
    return users.iloc[id]['Latitude']

def map_long_from_users(id):
    return users.iloc[id]['Longitude']

online['Latitude'] = online.User.map(map_lat_from_users)
online['Longitude'] = online.User.map(map_long_from_users)

C:\Users\haelkordi\AppData\Local\Temp\ipykernel_23076\964028055.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  online['Latitude'] = online.User.map(map_lat_from_users)
C:\Users\haelkordi\AppData\Local\Temp\ipykernel_23076\964028055.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  online['Longitude'] = online.User.map(map_long_from_users)


In [54]:
online.drop(columns=['Merchant City', 'Merchant State', 'Zip']).to_csv('./Data/AdjustedData/online.csv', index=False)

# Handle NaN Zip codes of non online transaction

In [22]:
nan_not_online = trans[(trans.Zip.isna()) & (trans['Merchant City'] != 'ONLINE')]

In [23]:
nan_not_online['loc'] = nan_not_online['Merchant City'] + ' ' + nan_not_online['Merchant State']

C:\Users\haelkordi\AppData\Local\Temp\ipykernel_23076\1208842936.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nan_not_online['loc'] = nan_not_online['Merchant City'] + ' ' + nan_not_online['Merchant State']


In [24]:
nan_not_online

,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?,loc
407,0,0,2003,2,24,13:02,$7.52,Swipe Transaction,-727612092139916043,Zurich,Switzerland,NaN,5411,NaN,No,Zurich Switzerland
408,0,0,2003,2,25,06:46,$36.97,Swipe Transaction,-4816289482172287511,Zurich,Switzerland,NaN,5411,NaN,No,Zurich Switzerland
906,0,0,2004,7,22,06:32,$38.26,Swipe Transaction,3510944833163794547,Tallinn,Estonia,NaN,5912,NaN,No,Tallinn Estonia
907,0,0,2004,7,23,20:05,$75.38,Swipe Transaction,8793545055147237096,Tallinn,Estonia,NaN,5812,NaN,No,Tallinn Estonia
908,0,0,2004,7,25,13:08,$16.20,Swipe Transaction,2027553650310142703,Tallinn,Estonia,NaN,5541,NaN,No,Tallinn Estonia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24385846,1999,1,2019,6,26,07:43,$54.40,Chip Transaction,8467004869390085980,Buenos Aires,Argentina,NaN,4121,NaN,No,Buenos Aires Argentina
24385847,1999,1,2019,6,26,11:38,$241.45,Swipe Transaction,-8566951830324093739,Buenos Aires,Argentina,NaN,3640,NaN,No,Buenos Aires Argentina
24385848,1999,1,2019,6,26,11:58,$-103.00,Swipe Transaction,-8566951830324093739,Buenos Aires,Argentina,NaN,3640,NaN,No,Buenos Aires Argentina
24385849,1999,1,2019,6,26,12:44,$118.29,Chip Transaction,190253443608377572,Buenos Aires,Argentina,NaN,3359,NaN,No,Buenos Aires Argentina


In [31]:
loc2zip = pd.DataFrame()
loc2zip['loc'] = nan_not_online['loc']
loc2zip.drop_duplicates(inplace=True)
loc2zip.reset_index(drop=True, inplace=True)

In [33]:
loc2zip['Latitude'] = loc2zip['loc'].map(map_lat_from_loc)
loc2zip['Longitude'] = loc2zip['loc'].map(map_long_from_loc)

In [35]:
loc2zip[loc2zip['Latitude'].isna()]

,loc,Latitude,Longitude
29,Johannesberg South Africa,NaN,NaN
96,Pnomh Penh Cambodia,NaN,NaN
185,Bishek Kyrgyzstan,NaN,NaN
189,Dili East Timor (Timor-Leste),NaN,NaN


In [46]:
loc2zip_dict = {}
for l in loc2zip['loc'].unique():
    ll = {}
    if l == 'Johannesberg South Africa':
        ll['Latitude'] = -26.195
        ll['Longitude'] = 28.034
    elif l == 'Pnomh Penh Cambodia':
        ll['Latitude'] = 11.562
        ll['Longitude'] = 104.888
    elif l == 'Bishek Kyrgyzstan':
        ll['Latitude'] = 42.882
        ll['Longitude'] = 74.582
    elif l == 'Dili East Timor (Timor-Leste)':
        ll['Latitude'] = -8.556
        ll['Longitude'] = 125.560
    else:
        ll['Latitude'] = loc2zip[loc2zip['loc'] == l]['Latitude'].values[0]
        ll['Longitude'] = loc2zip[loc2zip['loc'] == l]['Longitude'].values[0]
    loc2zip_dict[l] = ll  

In [49]:
def map_lat_from_loc(loc):
    return loc2zip_dict[loc]['Latitude']

def map_long_from_loc(loc):
    return loc2zip_dict[loc]['Longitude']

nan_not_online['Latitude'] = nan_not_online['loc'].map(map_lat_from_loc)
nan_not_online['Longitude'] = nan_not_online['loc'].map(map_long_from_loc)

C:\Users\haelkordi\AppData\Local\Temp\ipykernel_23076\912311970.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nan_not_online['Latitude'] = nan_not_online['loc'].map(map_lat_from_loc)
C:\Users\haelkordi\AppData\Local\Temp\ipykernel_23076\912311970.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nan_not_online['Longitude'] = nan_not_online['loc'].map(map_long_from_loc)


In [52]:
nan_not_online.drop(columns=['Merchant City', 'Merchant State', 'Zip', 'loc']).to_csv('./Data/AdjustedData/nan_not_online.csv', index=False)

# Handle Zip codes of non online transactions

In [4]:
non_online = trans[(~trans.Zip.isna()) & (trans['Merchant City'] != 'ONLINE')]

In [5]:
non_online.Zip = non_online.Zip.astype(int).astype(str)

C:\Users\haelkordi\AppData\Local\Temp\ipykernel_3732\29739700.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_online.Zip = non_online.Zip.astype(int).astype(str)


In [57]:
zip2loc = pd.DataFrame()
zip2loc['Zip'] = non_online['Zip']
zip2loc.drop_duplicates(inplace=True)
zip2loc.reset_index(drop=True, inplace=True)

In [59]:
pgeo = pgeocode.Nominatim('us')

In [61]:
def map_lat_from_loc(zip):
    l = pgeo.query_postal_code(zip)
    return l['latitude']

def map_long_from_loc(zip):
    l = pgeo.query_postal_code(zip)
    return l['longitude']

zip2loc['Latitude'] = zip2loc['Zip'].map(map_lat_from_loc)
zip2loc['Longitude'] = zip2loc['Zip'].map(map_long_from_loc)

In [62]:
zip2loc.to_csv('./Data/zip2loc.csv', index=False)

In [6]:
zip2loc = pd.read_csv('./Data/zip2loc.csv')

In [7]:
zip2loc.dropna(axis=0, inplace=True)

In [8]:
zip2loc_dict = {}
for l in zip2loc['Zip'].unique():
    ll = {}
    ll['Latitude'] = zip2loc[zip2loc['Zip'] == l]['Latitude'].values[0]
    ll['Longitude'] = zip2loc[zip2loc['Zip'] == l]['Longitude'].values[0]
    zip2loc_dict[l] = ll  

In [ ]:
def map_lat_from_zip2loc(zip):
    return zip2loc_dict[zip]['Latitude'] if zip in zip2loc.keys() else np.nan

def map_long_from_zip2loc(zip):
    return zip2loc_dict[zip]['Longitude'] if zip in zip2loc.keys() else np.nan

non_online['Latitude'] = non_online['Zip'].map(map_lat_from_zip2loc)
non_online['Longitude'] = non_online['Zip'].map(map_long_from_zip2loc)

In [60]:
non_online_nans = non_online[non_online.Latitude.isna()]
non_online_nans

,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?,Latitude,Longitude
0,0,0,2002,9,1,06:21,$134.09,Swipe Transaction,3527213246127876953,La Verne,CA,91750,5300,NaN,No,NaN,NaN
1,0,0,2002,9,1,06:42,$38.48,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754,5411,NaN,No,NaN,NaN
2,0,0,2002,9,2,06:22,$120.34,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754,5411,NaN,No,NaN,NaN
3,0,0,2002,9,2,17:45,$128.95,Swipe Transaction,3414527459579106770,Monterey Park,CA,91754,5651,NaN,No,NaN,NaN
4,0,0,2002,9,3,06:23,$104.71,Swipe Transaction,5817218446178736267,La Verne,CA,91750,5912,NaN,No,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24386895,1999,1,2020,2,27,22:23,$-54.00,Chip Transaction,-5162038175624867091,Merrimack,NH,3054,5541,NaN,No,NaN,NaN
24386896,1999,1,2020,2,27,22:24,$54.00,Chip Transaction,-5162038175624867091,Merrimack,NH,3054,5541,NaN,No,NaN,NaN
24386897,1999,1,2020,2,28,07:43,$59.15,Chip Transaction,2500998799892805156,Merrimack,NH,3054,4121,NaN,No,NaN,NaN
24386898,1999,1,2020,2,28,20:10,$43.12,Chip Transaction,2500998799892805156,Merrimack,NH,3054,4121,NaN,No,NaN,NaN


In [61]:
non_online_nans['loc'] = non_online_nans['Merchant City'] + ' ' + non_online_nans['Merchant State']

In [ ]:
from tqdm import tqdm

loc = {}
for l in tqdm(non_online_nans['loc'].unique()):
    ll = geolocator.geocode(l)
    loc[l] = ll

In [34]:
geolocator = Nominatim(user_agent="geoapiExercises")

In [36]:
geolocator.geocode('Chicago Park')

Location(Chicago park, Vincennes, Knox County, Indiana, United States, (38.694994949999995, -87.50869204289171, 0.0))

In [38]:
erroneous = []

In [39]:
for l in tqdm(non_online_nans['loc'].unique()):
    if l in loc.keys():
        continue
    try:
        ll = geolocator.geocode(l)
        loc[l] = ll
    except:
        erroneous.append(l)

100%|██████████| 20135/20135 [58:09<00:00,  5.77it/s]  


In [56]:
loc['Chicago Park CA'] = geolocator.geocode('Chicago park')
loc['Shaw A F B SC'] = geolocator.geocode('Shaw AFB SC')

In [66]:
len(loc.keys())

20136

In [62]:
def map_lat_from_loc_1(l):
    l = loc[l]
    return l[-1][0] if l is not None else np.nan

def map_long_from_loc_1(l):
    l = loc[l]
    return l[-1][1] if l is not None else np.nan

non_online_nans['Latitude'] = non_online_nans['loc'].map(map_lat_from_loc_1)
non_online_nans['Longitude'] = non_online_nans['loc'].map(map_long_from_loc_1)

In [143]:
non_online_nans.to_csv('./Data/AdjustedData/non_online.csv', index=False)

In [70]:
rem = list(non_online_nans[non_online_nans.Latitude.isna()]['loc'].drop_duplicates())

In [125]:
rem

['Diboll TX',
 'Purdon TX',
 'Kopperl TX',
 'Colmesneil TX',
 'Ben Bolt TX',
 'Boling TX',
 'Yatahey NM',
 'Thrall TX',
 'Elm Mott TX',
 'Gause TX',
 'Marion Station MD',
 'Garciasville TX',
 'Saint Hedwig TX',
 'Etoile TX',
 'La Vernia TX',
 'Olton TX',
 'Ropesville TX',
 'Pineland TX',
 'Chillicothe TX',
 'Onalaska TX',
 'Glendale Luke Afb AZ',
 'Gordonville TX',
 'Snook TX',
 'La Fayette GA',
 'De Mossville KY',
 'China Spring TX',
 'Brookeland TX',
 'Smokerun PA',
 'Moscow TX',
 'Tolar TX',
 'Cookville TX',
 'Quemado TX',
 'Annona TX',
 'Collinsville TX',
 'Hillister TX',
 'U S A F Academy CO',
 'Telferner TX',
 'Iola TX',
 'Mullin TX',
 'Batesville TX',
 'Muenster TX',
 'Bon Wier TX',
 'Yancey TX',
 'San Ygnacio TX',
 'Fe Warren Afb WY',
 'Newark TX',
 'La Pryor TX',
 'Knippa TX',
 'Forestburg TX',
 'Spurger TX',
 'Bangs TX',
 'Naval Air Station/ Jrb TX',
 'Reynolds Station KY',
 'Mapleton Depot PA',
 'Poth TX',
 'J B P H H HI',
 'Orangefield TX',
 'Asherton TX',
 'Brookston TX',


In [107]:
rem_dict = {}
for l in rem:
    f = l.replace('Mc ', 'Mc').replace('A F B', 'AFB')
    if f != l:
        ll = geolocator.geocode(f)
        if ll is None:
            f = ' '.join(f.split(' ')[:-1])
            ll = geolocator.geocode(f)
        rem_dict[l] = ll

In [108]:
len(rem_dict.keys())

37

In [111]:
for k in rem_dict.keys():
    rem.remove(k)

In [109]:
rem_dict

{'Lackland A F B TX': Location(Country Inn & Suites, 6502, Enrique M Barrera Parkway, Westwood Village, San Antonio, Bexar County, 78227, United States, (29.4056425, -98.61735335834058, 0.0)),
 'Mc Rae AR': Location(McRae, White County, Arkansas, United States, (35.1137095, -91.8217004, 0.0)),
 'Mc Connellsburg PA': Location(McConnellsburg, Fulton County, Pennsylvania, United States, (39.932589, -77.9988906, 0.0)),
 'Mc Louth KS': Location(McLouth, Jefferson County, Kansas, United States, (39.1958333, -95.2083035, 0.0)),
 'Mc Camey TX': Location(McCamey, Upton County, 79752, United States, (31.1315474, -102.222393, 0.0)),
 'Fort Mc Coy FL': Location(Fort McCoy, Marion County, Florida, 34489, United States, (29.3649734, -81.9670322, 0.0)),
 'Mc Calla AL': Location(McCalla, Jefferson County, Alabama, United States, (33.3062319, -87.0241656, 0.0)),
 'Mc Nabb IL': Location(McNabb, Putnam County, Illinois, United States, (41.1769799, -89.2092522, 0.0)),
 'Mc Rae GA': Location(US 280;US 319;

In [ ]:
 'J B P H H HI'

In [142]:
geolocator.geocode('JBPHH HI')

In [124]:
loc['Lackland A F B TX'] = geolocator.geocode('Lackland AFB TX')
loc['Diboll TX'] = geolocator.geocode('Diboll')
loc['Mc Rae AR'] = geolocator.geocode('McRae AR')
loc['Mc Connellsburg PA'] = geolocator.geocode('Mc Connellsburg PA')
loc['Purdon TX'] = geolocator.geocode('Purdon')
loc['M C B H Kaneohe Bay HI'] = geolocator.geocode('Kaneohe Bay')

In [128]:
loc['U S A F Academy CO'] = geolocator.geocode('USAF Academy CO')

In [133]:
loc['Naval Air Station/ Jrb TX'] = geolocator.geocode('Naval Air Station TX')